In [151]:
import geopandas as gpd
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import geoalchemy2
import numpy as np

In [152]:
#все пешеходные дороги в дубае
road_pesh = gpd.read_file('gdf_edges__.geojson')
#индустриальные районы
industrial = gpd.read_file('industrial.geojson')
#границы дубая
boundary_dubai = gpd.read_file('boundary_dubai.geojson')
#границы дубая
kvartal_dubai = gpd.read_file('kvartal_dubai_without_link.gpkg')


In [155]:
road_pesh['geometry'] = road_pesh['geometry'].to_crs(3857)
industrial['geometry'] = industrial['geometry'].to_crs(3857)


road_pesh_intersects = gpd.sjoin(road_pesh, industrial, how='inner', predicate='intersects', lsuffix='left', rsuffix='right')
road_all = road_pesh.merge(road_pesh_intersects, how='left',  indicator=True)
#дороги в индустриальных районах
road_main = road_all[road_all['_merge'] == 'left_only']
road_main['geometry'].to_file('road_main_all.geojson')

#находим длину дорог 
road_main['distant'] = road_main['length'].round()

road_pesh_in_boundary = gpd.sjoin(road_main, boundary_dubai, how='inner', predicate='intersects', lsuffix='l', rsuffix='r')
road_pesh_in_boundary['distant'] = road_pesh_in_boundary['length'].round()

In [162]:
import networkx as nx
#создаем граф 
G = nx.from_pandas_edgelist(road_main, 'u', 'v', edge_attr='distant')

Расчет центральности

In [117]:
degree_centrality = nx.degree_centrality(G)
# Преобразование в DataFrame
df = pd.DataFrame.from_dict(degree_centrality, orient='index', columns=['degree'])
df.index.name = 'node'
df = df.reset_index()
road_pesh_in_boundary = road_pesh_in_boundary.merge(df, how = 'left', left_on = 'u', right_on = 'node')

road_main = road_main.merge(df, how = 'left', left_on = 'u', right_on = 'node')
road_main[['u', 'v', 'geometry', 'length', 'degree_centrality', 'eigenvector_centrality']].to_file('road_centrality.geojson')

In [168]:
# Проверка связности графа
is_connected = nx.is_connected(G)

if is_connected:
    print("Граф связный.")
else:
    print("Граф несвязный.")

Граф несвязный.


In [169]:
bridges = list(nx.bridges(G))

# Создание копии графа без мостов
G_no_bridges = G.copy()
G_no_bridges.remove_edges_from(bridges)

In [165]:
# Создание копии графа без мостов
G_no_bridges = G.copy()
G_no_bridges.remove_edges_from(bridges)

# # Поиск связных компонент
# connected_components = list(nx.connected_components(G_no_bridges))
# S = [G.subgraph(c).copy() for c in nx.connected_components(G_no_bridges)]

In [92]:
# Поиск связных компонент
connected_components_with_bridges = list(nx.connected_components(G))

In [93]:
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]

In [171]:
# Поиск связных компонент
connected_components_with_bridges = list(nx.connected_components(G))

S = [G.subgraph(c).copy() for c in nx.connected_components(G_no_bridges)]

# Создание пустого датафрейма
combined_df_bridge = pd.DataFrame(columns=['source', 'target', 'Subgraph'])

# Объединение датафреймов в один
for i, subgraph in enumerate(S):
    # Преобразование подграфа в датафрейм
    df = pd.DataFrame(list(subgraph.edges()), columns=['source', 'target'])
    # Добавление столбца с идентификатором подграфа
    df['Subgraph'] = i
    # Объединение с общим датафреймом
    combined_df_bridge = pd.concat([combined_df_bridge, df], axis=0, ignore_index=True)

combined_df_bridge_filter = combined_df_bridge['Subgraph'].value_counts().reset_index().rename(columns={"index": "subgraph", "Subgraph": "count"})\
[combined_df_bridge['Subgraph'].value_counts().reset_index().rename(columns={"index": "subgraph", "Subgraph": "count"})['count'] > 1000]


road = combined_df_bridge_filter.merge(road_main, left_on = ['source', 'target'], right_on = ['u', 'v'])
road = gpd.GeoDataFrame(road, geometry = road.geometry, crs="EPSG:3857" )
road['Subgraph'] = road['Subgraph'].astype(int)
road[['u', 'v', 'geometry', 'Subgraph']].to_file('road_subgraph_all_bridge_2.geojson')


In [172]:
combined_df_bridge_filter = combined_df_bridge['Subgraph'].value_counts().reset_index().rename(columns={"index": "subgraph", "Subgraph": "count"})\
[combined_df_bridge['Subgraph'].value_counts().reset_index().rename(columns={"index": "subgraph", "Subgraph": "count"})['count'] > 1000]


In [173]:
combined_df_bridge_filter = combined_df_bridge.merge(combined_df_bridge_filter, left_on = 'Subgraph', right_on = 'subgraph')

In [ ]:
road[:len(road/3)]

df.iloc[:int(len(df)/3)]

In [140]:
road_itog = road[['u', 'v', 'geometry', 'Subgraph']]

In [141]:
# road_itog


road_itog_33 = road_itog.iloc[:int(len(road_itog)/3)]
road_itog_66 = road_itog.iloc[int(len(road_itog)/3):int(len(road_itog)*2/3)]
road_itog_100 = road_itog.iloc[int(len(road_itog)*2/3):]

In [143]:
roads = gpd.read_file('road_subgraph_all.geojson')

road_33 = roads.iloc[:int(len(roads)/3)]
road_66 = roads.iloc[int(len(roads)/3):int(len(roads)*2/3)]
road_100 = roads.iloc[int(len(roads)*2/3):]

road_33.to_file('road_33.geojson')
road_66.to_file('road_66.geojson')
road_100.to_file('road_100.geojson')

In [142]:
road_itog_33.to_file('road_itog_33.geojson')
road_itog_66.to_file('road_itog_66.geojson')
road_itog_100.to_file('road_itog_100.geojson')

In [ ]:
road[['u', 'v', 'geometry', 'Subgraph']].to_file('road_subgraph_all_bridge.geojson')

In [96]:
road = gpd.GeoDataFrame(road, geometry = road.geometry, crs="EPSG:3857" )

In [105]:
road['Subgraph'] = road['Subgraph'].astype(int)

In [97]:
road[['u', 'v', 'geometry', 'Subgraph']].to_file('road_subgraph_all.geojson')

In [107]:
road.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 874165 entries, 0 to 874164
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   source          874165 non-null  object  
 1   target          874165 non-null  object  
 2   Subgraph        874165 non-null  int32   
 3   u               874165 non-null  int64   
 4   v               874165 non-null  int64   
 5   key             874165 non-null  int64   
 6   highway         874165 non-null  object  
 7   length          874165 non-null  float64 
 8   lanes           87116 non-null   object  
 9   geometry        874165 non-null  geometry
 10  index_right     0 non-null       float64 
 11  area            0 non-null       float64 
 12  part_ind        0 non-null       object  
 13  urb_morph       0 non-null       object  
 14  transect        0 non-null       object  
 15  urb_str         0 non-null       object  
 16  urban_centres   0 non-null    

In [111]:
road['distant'] = road['length'].round()

In [110]:
road[road['Subgraph'] == 389]

,source,target,Subgraph,u,v,key,highway,length,lanes,geometry,...,part_ind,urb_morph,transect,urb_str,urban_centres,urb_env_types,Street pattern,Street front,Function,_merge
215782,8937537537,9983514389,389,8937537537,9983514389,0,service,3.463,NaN,"LINESTRING (6154544.899 2898705.815, 6154543.7...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
215783,8937537537,8938969844,389,8937537537,8938969844,0,service,15.983,NaN,"LINESTRING (6154544.899 2898705.815, 6154532.3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
215784,8937537537,9983514424,389,8937537537,9983514424,0,service,0.045,NaN,"LINESTRING (6154544.899 2898705.815, 6154544.9...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
215785,8937537538,8937537539,389,8937537538,8937537539,0,service,2.075,NaN,"LINESTRING (6154537.363 2898732.620, 6154535.3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
215786,8937537538,8937537547,389,8937537538,8937537547,0,service,2.086,NaN,"LINESTRING (6154537.363 2898732.620, 6154539.6...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326360,10053222361,10053222363,389,10053222361,10053222363,0,residential,2.883,NaN,"LINESTRING (6155151.190 2907131.462, 6155148.0...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
326361,10053222361,10053222362,389,10053222361,10053222362,0,residential,2.873,NaN,"LINESTRING (6155151.190 2907131.462, 6155154.2...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
326362,10053222364,10053222365,389,10053222364,10053222365,0,residential,1.761,NaN,"LINESTRING (6155167.854 2907137.247, 6155169.5...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
326363,10053222370,10053222368,389,10053222370,10053222368,0,residential,8.180,NaN,"LINESTRING (6155243.118 2907221.326, 6155246.8...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [112]:
G_389 = nx.from_pandas_edgelist(road[road['Subgraph'] == 389], 'u', 'v', edge_attr='distant')
# load_centrality = nx.load_centrality(G, cutoff=None, normalized=True)
edge_betweenness_centrality = nx.edge_betweenness_centrality(G_389, k=None, normalized=True)

KeyboardInterrupt: 